In [2]:
# ===========================================================
# Rozdz. 4.5 — Random Forest (pełny pipeline uczący)
# ===========================================================
import os, numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    roc_auc_score, average_precision_score, brier_score_loss, log_loss, roc_curve
)
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.inspection import permutation_importance

# ---------- ścieżki / artefakty ----------
ART = "artifacts_45_rf"
os.makedirs(ART, exist_ok=True)

# Ekonomia decyzji (dostosuj do realiów)
PROFIT_GOOD = 1_000
LOSS_BAD   = -5_000

# Walidacja czasowa
N_SPLITS_TIME = 3
N_BINS_CALIB  = 10
RANDOM_STATE  = 42

# ---------- 1) dane ----------
SNAP_PATH = Path("C:/Users/lukasz.wrobel/Desktop/PRACA MAGISTERSKA/pliki/artifacts/artifacts/engineered_snapshot.csv")
if not SNAP_PATH.exists():
    SNAP_PATH = Path("engineered_snapshot.csv")

df = pd.read_csv(SNAP_PATH)
if "issue_d" in df.columns:
    df["issue_d"] = pd.to_datetime(df["issue_d"], errors="coerce")

assert "loan_status_bin" in df.columns, "Brak kolumny 'loan_status_bin' w snapshotcie."
df["loan_status_bin"] = pd.to_numeric(df["loan_status_bin"], errors="coerce")
df = df.loc[df["loan_status_bin"].isin([0,1])].copy()

# y jako Series (zachowuje index -> później .loc)
y = df["loan_status_bin"].astype("int8")

# sanity — NaN/Inf w cechach
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# listy cech (bez gołych datetime)
feature_cols = [c for c in df.columns if c != "loan_status_bin" and not pd.api.types.is_datetime64_any_dtype(df[c])]
num_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df[c])]
cat_cols = [c for c in feature_cols if pd.api.types.is_object_dtype(df[c]) or pd.api.types.is_categorical_dtype(df[c])]
print(f"#kolumn num: {len(num_cols)}, kat: {len(cat_cols)}")

# ---------- 2) helpery ----------
def time_blocks(frame: pd.DataFrame, date_col="issue_d", n_splits=N_SPLITS_TIME):
    """Zwraca listę (train_idx, valid_idx) rosnących bloków czasowych (po miesiącach)."""
    if date_col not in frame.columns or frame[date_col].isna().all():
        # fallback 80/20 bez czasu
        idx = frame.index.to_numpy()
        cut = int(len(idx)*0.8)
        return [(idx[:cut], idx[cut:])]
    months = frame[date_col].dt.to_period("M").astype(str)
    uniq = np.array(sorted(months.dropna().unique()))
    if len(uniq) < n_splits:
        n_splits = max(2, len(uniq))
    chunks = np.array_split(uniq, n_splits)
    pairs = []
    for i in range(1, len(chunks)):
        tr_m = np.concatenate(chunks[:i])
        va_m = chunks[i]
        tr_idx = frame.index[months.isin(tr_m)]
        va_idx = frame.index[months.isin(va_m)]
        if len(tr_idx) and len(va_idx):
            pairs.append((tr_idx, va_idx))
    return pairs

def ks_score(y_true, y_prob):
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    return float(np.max(tpr - fpr))

def ece_score(y_true, y_prob, n_bins=20):
    bins = np.linspace(0,1,n_bins+1)
    idx = np.digitize(y_prob, bins) - 1
    ece = 0.0
    for b in range(n_bins):
        m = (idx == b)
        if m.sum()==0: 
            continue
        ece += m.mean() * abs(y_prob[m].mean() - y_true[m].mean())
    return float(ece)

def decile_table(y_true, y_prob, deciles=10):
    d = pd.DataFrame({"y": y_true, "p": y_prob}).sort_values("p", ascending=False).reset_index(drop=True)
    d["decile"] = pd.qcut(d.index, q=deciles, labels=False) + 1
    tab = d.groupby("decile").agg(
        n=("y","size"),
        bad=("y","sum"),
        good=("y", lambda s: (1-s).sum()),
        prob_mean=("p","mean")
    ).reset_index()
    tab["bad_rate"] = tab["bad"]/tab["n"]
    total_bad, total_good = tab["bad"].sum(), tab["good"].sum()
    tab["cum_bad"]  = tab["bad"].cumsum()/max(total_bad,1)
    tab["cum_good"] = tab["good"].cumsum()/max(total_good,1)
    tab["ks"] = (tab["cum_bad"] - tab["cum_good"]).abs()
    return tab

def profit_curve(y_true, y_prob, profit_good=PROFIT_GOOD, loss_bad=LOSS_BAD, steps=201):
    taus = np.linspace(0,1,steps)
    ev = []
    for t in taus:
        acc = y_prob < t
        tg = ((y_true==0) & acc).sum()
        tb = ((y_true==1) & acc).sum()
        ev.append(tg*profit_good + tb*loss_bad)
    return taus, np.array(ev)

# ---------- 3) preprocessing ----------
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)

num_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="median", add_indicator=True))  # RF nie wymaga skalowania
])
cat_pipe = Pipeline([
    ("imp", SimpleImputer(strategy="most_frequent")),
    ("ohe", ohe)
])
pre = ColumnTransformer(
    [("num", num_pipe, num_cols),
     ("cat", cat_pipe, cat_cols)],
    remainder="drop",
    verbose_feature_names_out=False
)

rf_base = RandomForestClassifier(
    n_estimators=500,
    max_depth=None,
    min_samples_leaf=100,
    class_weight="balanced",
    n_jobs=-1,
    random_state=RANDOM_STATE
)

rf_pipe = Pipeline([
    ("pre", pre),
    ("clf", rf_base)
])

# ---------- 4) walidacja czasowa + siatka hiperparametrów ----------
param_grid = {
    "clf__n_estimators": [100, 200],
    "clf__max_depth": [None, 10, 14],
    "clf__min_samples_leaf": [100, 200],
    "clf__max_features": ["sqrt", "log2", None]
}

folds = time_blocks(df, "issue_d", n_splits=N_SPLITS_TIME)
grid_results, last = [], {}
best_auc, best_params = -np.inf, None

for n_est in param_grid["clf__n_estimators"]:
    for max_depth in param_grid["clf__max_depth"]:
        for min_leaf in param_grid["clf__min_samples_leaf"]:
            for mfeat in param_grid["clf__max_features"]:
                rf_pipe.set_params(
                    clf__n_estimators=n_est,
                    clf__max_depth=max_depth,
                    clf__min_samples_leaf=min_leaf,
                    clf__max_features=mfeat
                )
                fold_aucs = []
                for tr_idx, va_idx in folds:
                    Xtr, ytr = df.loc[tr_idx, :], y.loc[tr_idx]
                    Xva, yva = df.loc[va_idx, :], y.loc[va_idx]
                    rf_pipe.fit(Xtr, ytr)
                    p = rf_pipe.predict_proba(Xva)[:,1]
                    fold_aucs.append(roc_auc_score(yva, p))
                    last = {"Xtr":Xtr, "ytr":ytr, "Xva":Xva, "yva":yva, "pva":p}
                mean_auc = float(np.mean(fold_aucs))
                grid_results.append({
                    "n_estimators": n_est, "max_depth": max_depth,
                    "min_samples_leaf": min_leaf, "max_features": mfeat,
                    "AUC_mean": mean_auc
                })
                if mean_auc > best_auc:
                    best_auc, best_params = mean_auc, (n_est, max_depth, min_leaf, mfeat)

cv_grid = pd.DataFrame(grid_results).sort_values("AUC_mean", ascending=False)
cv_grid.to_csv(f"{ART}/cv_grid_rf.csv", index=False)
print("Najlepsze parametry:", best_params, "AUC_mean=", round(best_auc,4))

# ustaw najlepsze i policz pełny zestaw metryk
rf_pipe.set_params(
    clf__n_estimators=600,
    clf__max_depth=best_params[1],
    clf__min_samples_leaf=best_params[2],
    clf__max_features=best_params[3]
)

metrics = []
for tr_idx, va_idx in folds:
    Xtr, ytr = df.loc[tr_idx, :], y.loc[tr_idx]
    Xva, yva = df.loc[va_idx, :], y.loc[va_idx]
    rf_pipe.fit(Xtr, ytr)
    p = rf_pipe.predict_proba(Xva)[:,1]
    metrics.append({
        "AUC": roc_auc_score(yva, p),
        "PR_AUC": average_precision_score(yva, p),
        "KS": ks_score(yva, p),
        "Brier": brier_score_loss(yva, p),
        "LogLoss": log_loss(yva, p, labels=[0,1]),
        "ECE": ece_score(yva, p)
    })
    last = {"Xtr":Xtr, "ytr":ytr, "Xva":Xva, "yva":yva, "pva":p}

cv_results = pd.DataFrame(metrics)
cv_results.to_csv(f"{ART}/cv_fold_metrics_rf.csv", index=False)
cv_mean = cv_results.mean()
cv_mean.to_csv(f"{ART}/cv_metrics_mean_rf.csv", header=False)
print("Średnie metryki CV (RF):\n", cv_mean.round(4))

# ---------- 5) ROC i kalibracja (ostatni fold) ----------
fpr, tpr, _ = roc_curve(last["yva"], last["pva"])
plt.figure(figsize=(5,4))
plt.plot(fpr, tpr, label=f"AUC={roc_auc_score(last['yva'],last['pva']):.3f}")
plt.plot([0,1],[0,1],"--")
plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC — Random Forest (ostatni fold)")
plt.legend(); plt.tight_layout(); plt.savefig(f"{ART}/roc_last_fold_rf.png", dpi=160); plt.close()

frac_pos, mean_pred = calibration_curve(last["yva"], last["pva"], n_bins=N_BINS_CALIB, strategy="quantile")
plt.figure(figsize=(5,4))
plt.plot(mean_pred, frac_pos, marker="o")
plt.plot([0,1],[0,1],"--")
plt.xlabel("Przewidziana PD"); plt.ylabel("Zaobserwowana stopa defaultu")
plt.title("Kalibracja — Random Forest (ostatni fold)")
plt.tight_layout(); plt.savefig(f"{ART}/calibration_last_fold_rf.png", dpi=160); plt.close()

# ---------- 6) Krzywa zysku + próg ----------
taus, ev = profit_curve(last["yva"], last["pva"], PROFIT_GOOD, LOSS_BAD, steps=201)
best_tau = float(taus[int(ev.argmax())])
pd.DataFrame({"tau":taus, "expected_profit":ev}).to_csv(f"{ART}/profit_curve_last_fold_rf.csv", index=False)

plt.figure(figsize=(6,4))
plt.plot(taus, ev); plt.axvline(best_tau, ls="--", label=f"tau*={best_tau:.3f}")
plt.xlabel("Próg akceptacji (p < tau)"); plt.ylabel("Oczekiwany zysk")
plt.title("Krzywa zysku — Random Forest (ostatni fold)")
plt.legend(); plt.tight_layout(); plt.savefig(f"{ART}/profit_curve_last_fold_rf.png", dpi=160); plt.close()

# ---------- 7) Test OOT (ostatni miesiąc) + kalibracja isotonic ----------
if "issue_d" in df.columns and df["issue_d"].notna().any():
    months = df["issue_d"].dt.to_period("M").astype(str)
    uniq = np.array(sorted(months.dropna().unique()))
    oot_mask = (months == uniq[-1])
    train_mask = ~oot_mask
else:
    idx = df.index.to_numpy()
    cut = int(len(idx)*0.8)
    train_mask = np.zeros(len(idx), dtype=bool); train_mask[:cut] = True
    oot_mask = ~train_mask

X_train, y_train = df.loc[train_mask, :], y.loc[train_mask]
X_oot,   y_oot   = df.loc[oot_mask,   :], y.loc[oot_mask]

rf_pipe.fit(X_train, y_train)
calibrated = CalibratedClassifierCV(rf_pipe, cv="prefit", method="isotonic")
calibrated.fit(last["Xva"], last["yva"])
p_oot = calibrated.predict_proba(X_oot)[:,1]

oot_metrics = {
    "AUC": roc_auc_score(y_oot, p_oot),
    "PR_AUC": average_precision_score(y_oot, p_oot),
    "KS": ks_score(y_oot, p_oot),
    "Brier": brier_score_loss(y_oot, p_oot),
    "LogLoss": log_loss(y_oot, p_oot, labels=[0,1]),
    "ECE": ece_score(y_oot, p_oot)
}
pd.Series(oot_metrics).to_csv(f"{ART}/oot_metrics_rf.csv", header=False)
print("\nMetryki OOT (RF):\n", pd.Series(oot_metrics).round(4))

# ---------- 8) Tabela decylowa + KS (OOT) ----------
dec_tab = decile_table(y_oot, p_oot, deciles=10)
dec_tab.to_csv(f"{ART}/decile_table_oot_rf.csv", index=False)

plt.figure(figsize=(6,4))
plt.plot(dec_tab["decile"], dec_tab["ks"], marker="o")
plt.xlabel("Decyl (1 = najwyższe ryzyko)"); plt.ylabel("KS")
plt.title("KS po decylach — Random Forest (OOT)")
plt.tight_layout(); plt.savefig(f"{ART}/ks_by_decile_oot_rf.png", dpi=160); plt.close()

# ---------- 9) Ważność cech: Gini + Permutation ----------
# nazwy cech po preprocesingu
pre_fitted = rf_pipe.named_steps["pre"].fit(X_train)
feat_names = pre_fitted.get_feature_names_out()

# Gini importance
fitted_rf = rf_pipe.named_steps["clf"]
imp = getattr(fitted_rf, "feature_importances_", None)
if imp is not None and len(imp) == len(feat_names):
    imp_df = pd.DataFrame({"feature": feat_names, "importance_gini": imp}).sort_values("importance_gini", ascending=False)
    imp_df.to_csv(f"{ART}/rf_feature_importance_gini.csv", index=False)

    plt.figure(figsize=(8,6))
    top = imp_df.head(15)[::-1]
    plt.barh(top["feature"], top["importance_gini"])
    plt.title("Random Forest — TOP 15 ważności (Gini)")
    plt.tight_layout(); plt.savefig(f"{ART}/rf_feature_importance_gini_top15.png", dpi=160); plt.close()

# Permutation importance (na walidacji z ostatniego foldu)
# załóżmy, że rf_pipe = Pipeline([("preprocess", pre), ("clf", rf)])
Xva_raw = last["Xva"]      # DataFrame z oryginalnymi kolumnami
yva     = last["yva"]
perm = permutation_importance(rf_pipe, Xva_raw, yva, n_repeats=5, random_state=RANDOM_STATE, n_jobs=-1)
# nazwy CECH PO transformacjach (OHE, passthrough, itd.)
# znajdź w pipelinie krok ColumnTransformer (nazwa kroku może być inna niż "preprocess")
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
# --- wyciągamy kroki z pipeline ---
# rf_pipe = Pipeline([("pre", pre), ("clf", rf)])  # przykładowa struktura
steps = dict(rf_pipe.named_steps)

# znajdź ColumnTransformer (nazwa kroku u Ciebie może być inna)
pre = None
for name, step in steps.items():
    if isinstance(step, ColumnTransformer):
        pre = step
        break
if pre is None:
    raise RuntimeError("Nie znaleziono ColumnTransformer w rf_pipe.")

# ostatni krok jako estymator (RandomForestClassifier)
clf = steps.get("clf", list(steps.values())[-1])

# --- dane walidacyjne w oryginalnej postaci ---
Xva_raw = last["Xva"]
yva     = last["yva"]

# --- transformacja walidacji dokładnie tym samym preprocessem ---
Xva_enc = pre.transform(Xva_raw)

# jeżeli wyjście jest rzadkie, a RF wymaga gęstej macierzy:
if hasattr(Xva_enc, "toarray"):
    try:
        Xva_enc = Xva_enc.toarray()
    except Exception:
        pass  # jeśli już jest gęsta, nic nie rób

# --- policz permutation importance na samym klasyfikatorze i zakodowanych cechach ---
perm = permutation_importance(
    clf, Xva_enc, yva,
    n_repeats=5, random_state=RANDOM_STATE, n_jobs=-1
)

# --- nazwy cech po transformacjach ---
if hasattr(pre, "get_feature_names_out"):
    feat_names = pre.get_feature_names_out()
else:
    feat_names = pre.get_feature_names()  # fallback dla starszego sklearn

# bezpieczeństwo: dopasuj długości
n = perm.importances_mean.shape[0]
if len(feat_names) != n:
    # spróbuj przyciąć / dopasować – lepiej niż wywalić się błędem
    feat_names = list(feat_names)[:n]

# --- zapis wyników ---
perm_df = pd.DataFrame({
    "feature": feat_names,
    "importance_perm_mean": perm.importances_mean,
    "importance_perm_std":  perm.importances_std
}).sort_values("importance_perm_mean", ascending=False)

perm_df.to_csv(f"{ART}/rf_feature_importance_permutation.csv", index=False)

plt.figure(figsize=(8,6))
top = perm_df.head(15)[::-1]
plt.barh(top["feature"], top["importance_perm_mean"])
plt.title("Random Forest — TOP 15 ważności (Permutation)")
plt.tight_layout(); plt.savefig(f"{ART}/rf_feature_importance_perm_top15.png", dpi=160); plt.close()

print(f"\nArtefakty zapisano w: {os.path.abspath(ART)}")


C:\Users\lukasz.wrobel\AppData\Local\Temp\ipykernel_22648\2943407291.py:53: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  cat_cols = [c for c in feature_cols if pd.api.types.is_object_dtype(df[c]) or pd.api.types.is_categorical_dtype(df[c])]


#kolumn num: 11, kat: 3
Najlepsze parametry: (200, None, 100, 'log2') AUC_mean= 0.7007
Średnie metryki CV (RF):
 AUC        0.7007
PR_AUC     0.3937
KS         0.2921
Brier      0.2120
LogLoss    0.6107
ECE        0.2198
dtype: float64

Metryki OOT (RF):
 AUC        0.7013
PR_AUC     0.4592
KS         0.2970
Brier      0.1833
LogLoss    0.5444
ECE        0.0137
dtype: float64

Artefakty zapisano w: c:\Users\lukasz.wrobel\Desktop\PRACA MAGISTERSKA\pliki\artifacts\artifacts_45_rf
